In [63]:
import numpy as np
import pandas as pd
import math
#
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt import objective_functions
#
import plotly.graph_objects as go
#
import importlib
import utilities.variables as variables
import utilities.utility as utility
import utilities.plots as plots
importlib.reload(variables)
importlib.reload(utility)
importlib.reload(plots)

<module 'utilities.plots' from '/Users/herbishtini/Documents/UNI/Master Thesis/sustainability_portfolio_optimisation/utilities/plots.py'>

In [3]:
# Read in price data
df = pd.read_csv('../../data/df_monthly_returns_complete.csv', index_col='Date')
df_pct = pd.read_csv('../../data/df_monthly_returns_complete_percentage.csv', index_col='Date')

### Train & Test split

In [4]:
# actual values
df_train = df.head(int(variables.ALL_YEARS_NR - 1) * 12)
df_test = df.tail(1 * 12)
# Percentage based values
df_pct_train = df_pct.head(int(variables.ALL_YEARS_NR - 1) * 12)
df_pct_test = df_pct.tail(1 * 12)

In [5]:
df_spread = utility.evenly_spaced_dataframe(df, 200)
df_train_spread = utility.evenly_spaced_dataframe(df_train, 200)

In [6]:
import cvxpy as cp

def get_portfolio_performance(df, file_name = "weights.csv", min_avg_return=variables.MIN_AVG_RETURN):
    # Calculate expected returns and sample covariance
    mu_0 = expected_returns.mean_historical_return(df, frequency=12)
    
    # Get only tickers with a mean historical return of at least 5% 
    optimal_tickers = mu_0[mu_0 > min_avg_return].index
    df_optimal = df[optimal_tickers]
    
    mu = expected_returns.mean_historical_return(df_optimal, frequency=12)
    S = risk_models.CovarianceShrinkage(df_optimal, frequency=12).ledoit_wolf() # Ledoit-Wolf shrinkage (df_optimal, frequency=12), # Exponential Covariance
    
    # Optimize for maximal Sharpe ratio
    ef = EfficientFrontier(mu, S, solver=cp.CLARABEL) # cp.ECOS
    ef.add_objective(objective_functions.L2_reg, gamma=0.1)
    
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()

    ef.save_weights_to_file(file_name)  # saves to file
    #
    p_mu, p_sigma, p_sharpe = ef.portfolio_performance(verbose=True)
    return df_optimal, cleaned_weights, mu, S, p_sigma, p_sharpe

def create_discrete_allocation(df, raw_weights, total_portfolio_value = 10000):
    latest_prices = get_latest_prices(df)

    da = DiscreteAllocation(raw_weights, latest_prices, total_portfolio_value=total_portfolio_value)
    allocation, leftover = da.greedy_portfolio()
    print("Discrete allocation:", allocation)
    print("Funds remaining: €{:.2f}".format(leftover))

### Discrete allocation 5 years

In [7]:
df_5y = df_train.tail(variables.FIVE_YEARS_NR * 12)
df_5y, raw_weights_5y, mu_5y, S_5y, sigma_5y, sharpe_5y = get_portfolio_performance(df_5y, "mpt_weights_5y.csv", min_avg_return=-0.5)
create_discrete_allocation(df_5y, raw_weights_5y)

/Users/herbishtini/anaconda3/envs/portfolio_optimisation/lib/python3.11/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


Expected annual return: 36.0%
Annual volatility: 10.6%
Sharpe Ratio: 3.20
Discrete allocation: {'ENPH': 4, 'SMCI': 1, 'LSCC': 5, '9697.T': 1, '6430.T': 1, 'LRN': 3, 'OMI': 6, 'NVDA': 2, 'AGYS': 1, 'O5G.DE': 92, 'HFG.DE': 2, 'LINC': 7, 'GTY.DE': 28, 'CLMB': 1, 'VSTO': 1, 'RERE': 6}
Funds remaining: €97.40


In [8]:
df_view_5y = pd.DataFrame.from_dict(raw_weights_5y, orient='index', columns=['max_sharpe_weight'])#.sort_values(by='weight', ascending=False)
# Extract volatilities (square root of diagonal elements)
df_view_5y['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_5y)), index=S_5y.columns).values
# Set annual returns
df_view_5y['avg_annual_return'] = mu_5y.values
#
df_view_5y['return_last_year'] = round(df_pct_test.prod() - 1, 2)
df_view_5y

,max_sharpe_weight,avg_annual_volatility,avg_annual_return,return_last_year
RS1.L,0.0,0.416109,0.057782,0.08
KE,0.0,0.470428,0.084197,-0.35
TEG.DE,0.0,0.445125,-0.088815,0.63
LEG.DE,0.0,0.423576,-0.049840,0.49
SCS,0.0,0.470052,-0.039691,0.22
...,...,...,...,...
KVHI,0.0,0.439677,-0.164630,-0.12
MOON.L,0.0,0.539367,-0.019952,0.22
NEO,0.0,0.547983,-0.079057,0.28
6055.T,0.0,0.460554,0.129850,-0.26


### Discrete allocation 15 years

In [9]:
df_15y = df_train.tail(variables.FIFTEEN_YEARS_NR * 12)
df_15y, raw_weights_15y, mu_15y, S_15y, sigma_15y, sharpe_15y = get_portfolio_performance(df_15y, "mpt_weights_15y.csv", min_avg_return=-0.5)
create_discrete_allocation(df_15y, raw_weights_15y)

/Users/herbishtini/anaconda3/envs/portfolio_optimisation/lib/python3.11/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


Expected annual return: 21.9%
Annual volatility: 5.9%
Sharpe Ratio: 3.38
Discrete allocation: {'SRT3.DE': 1, 'JDG.L': 1, 'NVDA': 1, 'AOF.DE': 1, 'NFLX': 1, 'SLP': 1, 'NXU.DE': 1, 'COK.DE': 1, 'NSSC': 1, 'GFT.DE': 1, 'MITK': 2, 'ECV.DE': 1, 'ADV.DE': 1, 'EVT.DE': 1, 'TAL': 2, 'TEG.DE': 1, 'FTK.DE': 1}
Funds remaining: €148.69


In [11]:
df_view_15y = pd.DataFrame.from_dict(raw_weights_15y, orient='index', columns=['max_sharpe_weight'])#.sort_values(by='weight', ascending=False)
# Set annual volatility
df_view_15y['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_15y)), index=S_15y.columns).values
# Actual returns of last year
df_view_15y['return_last_year'] = round(df_pct_test.prod() - 1, 2)
# Set annual returns
df_view_15y['avg_annual_return'] = mu_15y.values
df_view_15y

,max_sharpe_weight,avg_annual_volatility,return_last_year,avg_annual_return
RS1.L,0.00129,0.730134,0.08,0.130982
KE,0.00000,0.733732,-0.35,0.056930
TEG.DE,0.00146,0.733163,0.63,0.161162
LEG.DE,0.00000,0.729028,0.49,0.049242
SCS,0.00000,0.737874,0.22,0.048513
...,...,...,...,...
KVHI,0.00000,0.736296,-0.12,-0.009156
MOON.L,0.00000,0.734889,0.22,-0.004187
NEO,0.00119,0.745896,0.28,0.202069
6055.T,0.00573,0.735968,-0.26,0.286426


In [78]:
import utilities.plots as plots
importlib.reload(plots)

plots.plot_diff(df_view_15y.index.tolist(), df_view_15y['avg_annual_return'].values, df_view_15y['return_last_year'].values)

#### Efficient Frontier @TODO finish line

In [21]:
plots.plot_efficient_frontier(df_view_15y.index.tolist(), df_view_15y['avg_annual_return'].values, df_view_15y['avg_annual_volatility'].values, mu_15y, S_15y)